# Cellule 1: Installation des dépendances Unsloth

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

# Cellule 2: Import des bibliothèques

**Important:** This notebook requires a GPU to run. Please make sure you are running this in a GPU-accelerated environment.

In [ ]:
import os
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import (
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Cellule 3: Configuration du modèle et du dataset

In [ ]:
# Modèle de base que nous allons fine-tuner
model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
# Chemin vers votre dataset
dataset_file = "/content/haproxy_dataset_qa.jsonl"
# Nouveau nom pour notre modèle fine-tuné
new_model_name = "Qwen3-14B-unsloth-bnb-4bit-haproxy-expert"

# Cellule 4: Chargement du dataset

In [ ]:
dataset = load_dataset("json", data_files=dataset_file, split="train")
print(f"Dataset chargé avec {len(dataset)} exemples.")

# Cellule 5: Chargement du modèle et du tokenizer (via Unsloth)

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

# Chargement du modèle avec Unsloth
logging.info("Début du chargement du modèle...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=8192,
    dtype=None,
    load_in_4bit=True,
)
logging.info("Modèle chargé avec succès.")

# Ajouter les modules LoRA au modèle
logging.info("Ajout des modules LoRA...")
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Cellule 6: Formatage du dataset pour le chat template

In [ ]:
def format_chat_template(example):
    # Créer un contexte enrichi avec le title et le content si présents
    context = f"Référence: {example['title']}\n\nDocumentation: {example['content']}\n\n" if 'title' in example and 'content' in example else ""
    
    message = [
        {"role": "user", "content": f"{context}Question: {example['question']}"},
        {"role": "assistant", "content": example["response"]}
    ]
    # L'apply_chat_template formate le message pour le modèle
    text = tokenizer.apply_chat_template(message, tokenize=False)
    return {"text": text}

# On applique le formatage à tout le dataset
dataset = dataset.map(format_chat_template)

# Cellule 7: Configuration des arguments d'entraînement (TrainingArguments)

In [ ]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.001,
    warmup_ratio=0.03,
    lr_scheduler_type="linear",
    optim="adamw_8bit",
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    seed=3407,
    report_to="none",
    output_dir="./results",
    remove_unused_columns=False,
)

# Cellule 8: Initialisation et lancement de l'entraînement

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=8192,
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
)

trainer.train()

# Cellule 9: Sauvegarde du modèle fine-tuné

In [ ]:
trainer.save_model(new_model_name)
tokenizer.save_pretrained(new_model_name)
print(f"Modèle fine-tuné sauvegardé sous le nom : {new_model_name}")

# Cellule 10: Test du modèle fine-tuné

In [ ]:
# Activer le mode inférence
FastLanguageModel.for_inference(model)

# Question de test
test_prompt = "Quelle est la directive 'bind' dans HAProxy et comment l'utiliser ?"
inputs = tokenizer([f"### User:\n{test_prompt}\n\n### Assistant:\n"], return_tensors="pt").to(model.device)

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        use_cache=True,
        temperature=0.7,
        do_sample=True,
    )
    response = tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[-1]:], skip_special_tokens=True)[0]
    print(response)